En este archivo agregamos la columna de "Temperatura" y de "Precipitacion" a la Base de Datos

**Importamos las paqueterias necesarias para el analisis.**

In [1]:
import pandas as pd
import glob
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Importamos las bases de datos por año**

In [3]:
SalesTotal = pd.read_pickle('/content/drive/MyDrive/DATATHON_DSC/Sales_Total/Sales_Total.h5')
SalesTotal

,Empresa,Region,Filial,PdVenta,KeyCliente,KeyFecha,KeyFolio,KeyTicket,Litros,Venta
225847,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Centro,SAN NICOLAS,E.C. RUIZ CORTINEZ 2,M00938,2017-01-01,Cf16R0,yyx7D8,2.0,13.0
111653,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,MONTEMORELOS,E.C. CRUZ VERDE,MM0143,2017-01-01,EH14hM,yvC3PX,3.0,26.0
111652,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,MONTEMORELOS,E.C. CRUZ VERDE,MM0143,2017-01-01,EH14hL,yvC3PW,5.0,40.0
111651,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,MONTEMORELOS,E.C. CRUZ VERDE,MM0143,2017-01-01,EH14hK,yvC3PV,6.0,53.0
111650,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,MONTEMORELOS,E.C. CRUZ VERDE,MM0143,2017-01-01,EH14hJ,yvC3PU,2.0,13.0
...,...,...,...,...,...,...,...,...,...,...
209781,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Centro,REYNOSA,ATQ. REY 06,RY0001,2021-12-31,3000-62B9-F62A-1000-1EE2,zVOADA,8.0,81.0
209780,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Centro,REYNOSA,ATQ. REY 23 (077),RY0001,2021-12-31,3001-72B9-F643-9000-83AC,zVOAD9,3.0,28.0
209779,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Centro,REYNOSA,ATQ. REY 23 (077),RY0001,2021-12-31,3001-72B9-F63F-9800-83AB,zVOAD7,13.0,134.0
209794,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Centro,REYNOSA,ATQ. REY 10,RY0001,2021-12-31,3004-F2B9-F382-7800-194D,zVOADX,3.0,31.0


**Seleccionamos solo el conjunto de datos que nos interesa.**

In [4]:
SalesTotal = SalesTotal[["Region", "Filial", "KeyCliente", "KeyFecha", "Litros", "Venta"]] 

**Quitamos los datos de Mayoreo ya que no corrresponden a ningun área geográfica.**

In [5]:
SalesTotal = SalesTotal.loc[SalesTotal['Region'] != 'Mayoreo']

In [6]:
SalesTotal

,Region,Filial,KeyCliente,KeyFecha,Litros,Venta
225847,Centro,SAN NICOLAS,M00938,2017-01-01,2.0,13.0
111653,Este,MONTEMORELOS,MM0143,2017-01-01,3.0,26.0
111652,Este,MONTEMORELOS,MM0143,2017-01-01,5.0,40.0
111651,Este,MONTEMORELOS,MM0143,2017-01-01,6.0,53.0
111650,Este,MONTEMORELOS,MM0143,2017-01-01,2.0,13.0
...,...,...,...,...,...,...
209781,Centro,REYNOSA,RY0001,2021-12-31,8.0,81.0
209780,Centro,REYNOSA,RY0001,2021-12-31,3.0,28.0
209779,Centro,REYNOSA,RY0001,2021-12-31,13.0,134.0
209794,Centro,REYNOSA,RY0001,2021-12-31,3.0,31.0


**Agrupamos respecto a la Fecha (KeyFecha) y la Filial simultaneámente.**

In [7]:
SalesTotalbyMonth = SalesTotal.groupby([SalesTotal.KeyFecha.dt.to_period('M'), 'Filial']).sum().reset_index()
SalesTotalbyMonth

,KeyFecha,Filial,Litros,Venta
0,2017-01,ALAMO,136123.0,1120651.0
1,2017-01,ALDAMA,51862.0,440557.0
2,2017-01,ALLENDE,94917.0,776635.0
3,2017-01,CALPULALPAN,67137.0,522378.0
4,2017-01,CD. VICTORIA,87200.0,715542.0
...,...,...,...,...
1337,2021-12,SAN NICOLAS,448734.0,5671409.0
1338,2021-12,SANTA CATARINA,108538.0,1374997.0
1339,2021-12,SOTO LA MARINA,25809.0,324083.0
1340,2021-12,TAMPICO,212424.0,2726534.0


**Agregamos el dictionario de  Filial a Estado y le asignamos una función. Fuente: Google Maps**

In [8]:
dict_filial_edo = {'ALAMO': 'VEZ',
'ALDAMA': 'CHI',
'ALLENDE': 'NL',
'CALPULALPAN': 'TLX',
'Cd Valles': 'SLP',	
'CD. VICTORIA': 'TMS',
'GALEANA': 'NL',
'HILDA G GARCIA': 'COA',
'JIMENEZ': 'TMS',
'LAREDO': 'TMS',
'LINARES': 'NL',
'MATEHUALA': 'SLP',
'MONCLOVA': 'COA',
'MONTEMORELOS': 'NL',
'POZA RICA': 'VEZ',
'REYNOSA': 'TMS',
'RIO VERDE': 'SLP',
'SALINASVICTORIA': 'NL',
'SALTILLO': 'COA',
'SAN FERNANDO': 'TMS',
'SAN LUIS POTOSI': 'SLP',
'SAN NICOLAS': 'NL',
'SAN RAFAEL': 'NL',
'SANTA CATARINA': 'NL',
'SOTO LA MARINA': 'TMS',
'TAMPICO': 'TMS',
'VARAGAS': 'COA'}

In [9]:
def Filial_Estado(x):

  return dict_filial_edo[x]

**Agregamos una nueva columna con la información del estado**

In [10]:
SalesTotalbyMonth['Estado'] = SalesTotalbyMonth.apply(lambda x: Filial_Estado(x['Filial']), axis=1)

In [11]:
SalesTotalbyMonth

,KeyFecha,Filial,Litros,Venta,Estado
0,2017-01,ALAMO,136123.0,1120651.0,VEZ
1,2017-01,ALDAMA,51862.0,440557.0,CHI
2,2017-01,ALLENDE,94917.0,776635.0,NL
3,2017-01,CALPULALPAN,67137.0,522378.0,TLX
4,2017-01,CD. VICTORIA,87200.0,715542.0,TMS
...,...,...,...,...,...
1337,2021-12,SAN NICOLAS,448734.0,5671409.0,NL
1338,2021-12,SANTA CATARINA,108538.0,1374997.0,NL
1339,2021-12,SOTO LA MARINA,25809.0,324083.0,TMS
1340,2021-12,TAMPICO,212424.0,2726534.0,TMS


**Agregamos el dictionario de Estado a Clave y le asignamos una función**

In [12]:
dict_edo_clave={
  "AGS": 0,
  "BC": 1,
  "BCS": 2,
  "CAM": 3,
  "COA": 4,
  "COL": 5,
  "CHP": 6,
  "CHI": 7,
  "DF": 8,
  "DUR": 9,
  "GUT": 10,
  "GUR": 11,
  "HIG": 12,
  "JAL": 13,
  "MEX": 14,
  "MCH": 15,
  "MOR": 16,
  "NYT": 17,
  "NL": 18,
  "OAX": 19,
  "PBL": 20,
  "QUE": 21,
  "QR":  22,
  "SLP": 23,
  "SIN": 24,
  "SON": 25,
  "TAB": 26,
  "TMS": 27,
  "TLX": 28,
  "VEZ": 29,
  "YUT": 30,
  "ZCS": 31,
  "NAC": 32
}

**Agregamos una nueva columna con la información de la clave 'KeyEstado'**

In [13]:
SalesTotalbyMonth['KeyEstado'] = SalesTotalbyMonth.apply(lambda x: dict_edo_clave[x['Estado']], axis=1)

In [14]:
SalesTotalbyMonth

,KeyFecha,Filial,Litros,Venta,Estado,KeyEstado
0,2017-01,ALAMO,136123.0,1120651.0,VEZ,29
1,2017-01,ALDAMA,51862.0,440557.0,CHI,7
2,2017-01,ALLENDE,94917.0,776635.0,NL,18
3,2017-01,CALPULALPAN,67137.0,522378.0,TLX,28
4,2017-01,CD. VICTORIA,87200.0,715542.0,TMS,27
...,...,...,...,...,...,...
1337,2021-12,SAN NICOLAS,448734.0,5671409.0,NL,18
1338,2021-12,SANTA CATARINA,108538.0,1374997.0,NL,18
1339,2021-12,SOTO LA MARINA,25809.0,324083.0,TMS,27
1340,2021-12,TAMPICO,212424.0,2726534.0,TMS,27


**Agregamos una nueva columna 'KeyAM' con la información de la Fecha, en el formato númerico {AñoMes}**

In [15]:
SalesTotalbyMonth['KeyAM'] = SalesTotalbyMonth['KeyFecha'].map(lambda x : str(x.year) + str(x.month).zfill(2)) 

In [16]:
SalesTotalbyMonth

,KeyFecha,Filial,Litros,Venta,Estado,KeyEstado,KeyAM
0,2017-01,ALAMO,136123.0,1120651.0,VEZ,29,201701
1,2017-01,ALDAMA,51862.0,440557.0,CHI,7,201701
2,2017-01,ALLENDE,94917.0,776635.0,NL,18,201701
3,2017-01,CALPULALPAN,67137.0,522378.0,TLX,28,201701
4,2017-01,CD. VICTORIA,87200.0,715542.0,TMS,27,201701
...,...,...,...,...,...,...,...
1337,2021-12,SAN NICOLAS,448734.0,5671409.0,NL,18,202112
1338,2021-12,SANTA CATARINA,108538.0,1374997.0,NL,18,202112
1339,2021-12,SOTO LA MARINA,25809.0,324083.0,TMS,27,202112
1340,2021-12,TAMPICO,212424.0,2726534.0,TMS,27,202112


#Temperatura

**Leemos el archivo 'Temperature.csv', que posee datos de temperatura promedio en grados Celsius (°C) en la Republica por Estado de Datos de la CONAGUA (https://smn.conagua.gob.mx/es/climatologia/temperaturas-y-lluvias/resumenes-mensuales-de-temperaturas-y-lluvias)**


In [17]:
Temperature_path = '/content/drive/MyDrive/DATATHON_DSC/Temperatura/Temperatura_cod.csv'
Temperatura = pd.read_csv(Temperature_path, low_memory=False, index_col=False, thousands=',')

**Hacemos una función que tome la Clave del Estado 'KeyEstado' y la información de la fecha 'KeyAM' y devuelve la temperatura**

In [18]:
def Temperature_function(x,y):
  
  return Temperatura[y][x]

**Creamos una nueva columna con la Temperatura.**

In [19]:
SalesTotalbyMonth['Temperatura'] = SalesTotalbyMonth[['KeyEstado','KeyAM']].apply(lambda x: Temperature_function(*x), axis=1)

In [20]:
SalesTotalbyMonth

,KeyFecha,Filial,Litros,Venta,Estado,KeyEstado,KeyAM,Temperatura
0,2017-01,ALAMO,136123.0,1120651.0,VEZ,29,201701,20.7
1,2017-01,ALDAMA,51862.0,440557.0,CHI,7,201701,11.7
2,2017-01,ALLENDE,94917.0,776635.0,NL,18,201701,16.8
3,2017-01,CALPULALPAN,67137.0,522378.0,TLX,28,201701,12.5
4,2017-01,CD. VICTORIA,87200.0,715542.0,TMS,27,201701,20.3
...,...,...,...,...,...,...,...,...
1337,2021-12,SAN NICOLAS,448734.0,5671409.0,NL,18,202112,19.0
1338,2021-12,SANTA CATARINA,108538.0,1374997.0,NL,18,202112,19.0
1339,2021-12,SOTO LA MARINA,25809.0,324083.0,TMS,27,202112,22.2
1340,2021-12,TAMPICO,212424.0,2726534.0,TMS,27,202112,22.2


**Guardamos como csv el nuevo Dataframe con la columna de Temperatura**

In [21]:
SalesTotalbyMonth.to_csv('/content/drive/MyDrive/DATATHON_DSC/Temperatura/SalesTotalbyMonth_withTemperature.csv')  

In [22]:
# !ls /content/drive/MyDrive/DATATHON_DSC/Temperatura/

#Lluvia

**Leemos el archivo 'LluviaTotal.csv', que posee datos de precipitacion acumulada mensual (mm) por Estado de Datos de la CONAGUA (https://smn.conagua.gob.mx/es/climatologia/temperaturas-y-lluvias/resumenes-mensuales-de-temperaturas-y-lluvias)**

In [23]:
Rain_path = '/content/drive/MyDrive/DATATHON_DSC/Lluvia/LluviaTotal.csv'
Lluvia = pd.read_csv(Rain_path, low_memory=False, index_col=False, thousands=',', encoding='latin-1')

In [24]:
Lluvia

,ENTIDAD,201701,201702,201704,201703,201705,201706,201707,201708,201709,...,202108,202109,202110,202111,202112,202100,202201,202202,202203,202204
0,AGUASCALIENTES,0.4,2.1,6.7,0.4,4.2,43.5,169.3,143.8,159.5,...,117.5,185.5,36.6,0.2,1.1,672.3,1.0,3.9,0.0,15.3
1,BAJA CALIFORNIA,53.6,44.9,1.4,0.0,7.9,0.2,2.9,7.1,9.6,...,7.5,3.8,3.0,0.7,37.3,99.1,1.4,8.7,13.8,0.7
2,BAJA CALIFORNIA SUR,8.4,18.4,0.5,0.1,1.6,1.9,12.1,68.3,59.0,...,53.5,53.7,6.6,3.3,23.7,173.5,0.3,0.2,0.1,0.0
3,CAMPECHE,16.1,6.9,9.1,65.1,7.0,363.6,169.2,202.8,229.8,...,147.7,230.4,138.7,52.4,45.8,1218.7,65.0,57.3,52.3,79.2
4,COAHUILA,2.6,11.1,15.1,23.0,40.6,22.6,37.8,45.1,108.3,...,43.0,24.1,12.0,16.2,5.0,246.5,2.6,1.3,0.1,14.2
5,COLIMA,0.0,2.8,1.2,0.0,1.7,197.4,287.4,560.3,528.0,...,705.1,443.7,187.0,0.2,0.2,2280.7,1.3,0.7,0.0,0.5
6,CHIAPAS,36.8,8.1,23.1,64.0,262.3,409.4,254.3,249.2,401.7,...,299.0,320.1,227.8,57.5,41.6,1807.2,74.5,78.3,30.3,84.4
7,CHIHUAHUA,12.3,8.1,6.1,3.4,10.4,20.6,153.3,122.1,58.8,...,105.4,46.1,7.7,11.2,24.0,410.6,0.1,3.8,0.2,1.7
8,DISTRITO FEDERAL,0.0,0.0,18.1,12.9,48.9,86.2,101.7,101.3,86.8,...,126.7,120.4,49.7,1.3,0.0,637.1,3.8,1.6,5.2,44.0
9,DURANGO,1.3,7.8,13.3,1.4,4.7,12.5,121.7,139.3,97.3,...,121.5,68.9,52.6,3.6,9.0,469.9,0.2,2.5,0.0,3.6


**Agregamos la columna de Estado al Dataframe de Lluvia**

In [25]:
Lluvia['Estado'] = Temperatura['ENTIDAD FEDERATIVA']

In [26]:
Lluvia

,ENTIDAD,201701,201702,201704,201703,201705,201706,201707,201708,201709,...,202109,202110,202111,202112,202100,202201,202202,202203,202204,Estado
0,AGUASCALIENTES,0.4,2.1,6.7,0.4,4.2,43.5,169.3,143.8,159.5,...,185.5,36.6,0.2,1.1,672.3,1.0,3.9,0.0,15.3,AGS
1,BAJA CALIFORNIA,53.6,44.9,1.4,0.0,7.9,0.2,2.9,7.1,9.6,...,3.8,3.0,0.7,37.3,99.1,1.4,8.7,13.8,0.7,BC
2,BAJA CALIFORNIA SUR,8.4,18.4,0.5,0.1,1.6,1.9,12.1,68.3,59.0,...,53.7,6.6,3.3,23.7,173.5,0.3,0.2,0.1,0.0,BCS
3,CAMPECHE,16.1,6.9,9.1,65.1,7.0,363.6,169.2,202.8,229.8,...,230.4,138.7,52.4,45.8,1218.7,65.0,57.3,52.3,79.2,CAM
4,COAHUILA,2.6,11.1,15.1,23.0,40.6,22.6,37.8,45.1,108.3,...,24.1,12.0,16.2,5.0,246.5,2.6,1.3,0.1,14.2,COA
5,COLIMA,0.0,2.8,1.2,0.0,1.7,197.4,287.4,560.3,528.0,...,443.7,187.0,0.2,0.2,2280.7,1.3,0.7,0.0,0.5,COL
6,CHIAPAS,36.8,8.1,23.1,64.0,262.3,409.4,254.3,249.2,401.7,...,320.1,227.8,57.5,41.6,1807.2,74.5,78.3,30.3,84.4,CHP
7,CHIHUAHUA,12.3,8.1,6.1,3.4,10.4,20.6,153.3,122.1,58.8,...,46.1,7.7,11.2,24.0,410.6,0.1,3.8,0.2,1.7,CHI
8,DISTRITO FEDERAL,0.0,0.0,18.1,12.9,48.9,86.2,101.7,101.3,86.8,...,120.4,49.7,1.3,0.0,637.1,3.8,1.6,5.2,44.0,DF
9,DURANGO,1.3,7.8,13.3,1.4,4.7,12.5,121.7,139.3,97.3,...,68.9,52.6,3.6,9.0,469.9,0.2,2.5,0.0,3.6,DUR


In [27]:
Lluvia = Lluvia[pd.core.indexes.base.Index(['Estado']).append(Lluvia.columns[:len(Lluvia.columns)-1])]

In [28]:
Lluvia

,Estado,ENTIDAD,201701,201702,201704,201703,201705,201706,201707,201708,...,202108,202109,202110,202111,202112,202100,202201,202202,202203,202204
0,AGS,AGUASCALIENTES,0.4,2.1,6.7,0.4,4.2,43.5,169.3,143.8,...,117.5,185.5,36.6,0.2,1.1,672.3,1.0,3.9,0.0,15.3
1,BC,BAJA CALIFORNIA,53.6,44.9,1.4,0.0,7.9,0.2,2.9,7.1,...,7.5,3.8,3.0,0.7,37.3,99.1,1.4,8.7,13.8,0.7
2,BCS,BAJA CALIFORNIA SUR,8.4,18.4,0.5,0.1,1.6,1.9,12.1,68.3,...,53.5,53.7,6.6,3.3,23.7,173.5,0.3,0.2,0.1,0.0
3,CAM,CAMPECHE,16.1,6.9,9.1,65.1,7.0,363.6,169.2,202.8,...,147.7,230.4,138.7,52.4,45.8,1218.7,65.0,57.3,52.3,79.2
4,COA,COAHUILA,2.6,11.1,15.1,23.0,40.6,22.6,37.8,45.1,...,43.0,24.1,12.0,16.2,5.0,246.5,2.6,1.3,0.1,14.2
5,COL,COLIMA,0.0,2.8,1.2,0.0,1.7,197.4,287.4,560.3,...,705.1,443.7,187.0,0.2,0.2,2280.7,1.3,0.7,0.0,0.5
6,CHP,CHIAPAS,36.8,8.1,23.1,64.0,262.3,409.4,254.3,249.2,...,299.0,320.1,227.8,57.5,41.6,1807.2,74.5,78.3,30.3,84.4
7,CHI,CHIHUAHUA,12.3,8.1,6.1,3.4,10.4,20.6,153.3,122.1,...,105.4,46.1,7.7,11.2,24.0,410.6,0.1,3.8,0.2,1.7
8,DF,DISTRITO FEDERAL,0.0,0.0,18.1,12.9,48.9,86.2,101.7,101.3,...,126.7,120.4,49.7,1.3,0.0,637.1,3.8,1.6,5.2,44.0
9,DUR,DURANGO,1.3,7.8,13.3,1.4,4.7,12.5,121.7,139.3,...,121.5,68.9,52.6,3.6,9.0,469.9,0.2,2.5,0.0,3.6


In [29]:
# SalesTotalbyMonth['KeyEstado'] = SalesTotalbyMonth.apply(lambda x: dict_edo_clave[x['Estado']], axis=1)

In [30]:
SalesTotalbyMonth

,KeyFecha,Filial,Litros,Venta,Estado,KeyEstado,KeyAM,Temperatura
0,2017-01,ALAMO,136123.0,1120651.0,VEZ,29,201701,20.7
1,2017-01,ALDAMA,51862.0,440557.0,CHI,7,201701,11.7
2,2017-01,ALLENDE,94917.0,776635.0,NL,18,201701,16.8
3,2017-01,CALPULALPAN,67137.0,522378.0,TLX,28,201701,12.5
4,2017-01,CD. VICTORIA,87200.0,715542.0,TMS,27,201701,20.3
...,...,...,...,...,...,...,...,...
1337,2021-12,SAN NICOLAS,448734.0,5671409.0,NL,18,202112,19.0
1338,2021-12,SANTA CATARINA,108538.0,1374997.0,NL,18,202112,19.0
1339,2021-12,SOTO LA MARINA,25809.0,324083.0,TMS,27,202112,22.2
1340,2021-12,TAMPICO,212424.0,2726534.0,TMS,27,202112,22.2


**Hacemos una función que tome la Clave del Estado 'KeyEstado' y la información de la fecha 'KeyAM' y devuelve la precipitación**

In [31]:
def Rain_function(x,y):
  
  return Lluvia[y][x]

**Creamos una nueva columna con la Precipitacion.**

In [32]:
SalesTotalbyMonth['Lluvia'] = SalesTotalbyMonth[['KeyEstado','KeyAM']].apply(lambda x: Rain_function(*x), axis=1)

In [33]:
SalesTotalbyMonth

,KeyFecha,Filial,Litros,Venta,Estado,KeyEstado,KeyAM,Temperatura,Lluvia
0,2017-01,ALAMO,136123.0,1120651.0,VEZ,29,201701,20.7,36.9
1,2017-01,ALDAMA,51862.0,440557.0,CHI,7,201701,11.7,12.3
2,2017-01,ALLENDE,94917.0,776635.0,NL,18,201701,16.8,10.2
3,2017-01,CALPULALPAN,67137.0,522378.0,TLX,28,201701,12.5,0.4
4,2017-01,CD. VICTORIA,87200.0,715542.0,TMS,27,201701,20.3,8.3
...,...,...,...,...,...,...,...,...,...
1337,2021-12,SAN NICOLAS,448734.0,5671409.0,NL,18,202112,19.0,12.0
1338,2021-12,SANTA CATARINA,108538.0,1374997.0,NL,18,202112,19.0,12.0
1339,2021-12,SOTO LA MARINA,25809.0,324083.0,TMS,27,202112,22.2,12.6
1340,2021-12,TAMPICO,212424.0,2726534.0,TMS,27,202112,22.2,12.6


**Guardamos como csv el nuevo Dataframe con la columna de Temperatura y de Precipitación**

In [34]:
SalesTotalbyMonth.to_csv('/content/drive/MyDrive/DATATHON_DSC/Lluvia/SalesTotalbyMonth_withTemperature_Rain.csv')  

In [35]:
# !ls /content/drive/MyDrive/DATATHON_DSC/Lluvia/

2017lluviatabla.csv  2021lluviatabla.csv
2018lluviatabla.csv  2022lluviatabla.csv
2019lluviatabla.csv  LluviaTotal.csv
2020lluviatabla.csv  SalesTotalbyMonth_withTemperature_Rain.csv


In [45]:
!jupyter nbconvert --to html '/content/drive/MyDrive/DATATHON_DSC/SalesTotalbyMonth_withTemperature_Rain.ipynb'

[NbConvertApp] Converting notebook /content/drive/MyDrive/DATATHON_DSC/SalesTotalbyMonth_withTemperature_Rain.ipynb to html
[NbConvertApp] Writing 420329 bytes to /content/drive/MyDrive/DATATHON_DSC/SalesTotalbyMonth_withTemperature_Rain.html
